## Lab 2_4 torch.nn.Linear - Classification (non-convolution)

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

In [3]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

Failed to download (trying next):
<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>



100%|██████████| 9.91M/9.91M [00:08<00:00, 1.21MB/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>



100%|██████████| 28.9k/28.9k [00:00<00:00, 127kB/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>



100%|██████████| 1.65M/1.65M [00:02<00:00, 755kB/s] 


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>



100%|██████████| 4.54k/4.54k [00:00<?, ?B/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [7]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

train_loader.dataset


Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

In [8]:
class SimpleNN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNN, self).__init__()

        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, X):
        A1 = F.relu(self.fc1(X))
        A2 = F.relu(self.fc2(A1))
        return self.fc3(A2)


In [9]:
input_dim = 28 * 28  # 28x28 pixels in MNIST images
hidden_dim = 128  # Number of neurons in the hidden layer
output_dim = 10  # 10 classes (digits 0-9)

In [10]:
model = SimpleNN(input_dim, hidden_dim, output_dim)
# Step 3: Define the loss function (cross-entropy) and optimizer (Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [11]:
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for data, target in train_loader:
        data = data.view(data.size(0), -1)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}')

Epoch 1/10, Loss: 0.36254907854552715
Epoch 2/10, Loss: 0.16844153797813952
Epoch 3/10, Loss: 0.12471005214310722
Epoch 4/10, Loss: 0.10233806121423206
Epoch 5/10, Loss: 0.08558879150593998
Epoch 6/10, Loss: 0.07655535912317937
Epoch 7/10, Loss: 0.06546427367968974
Epoch 8/10, Loss: 0.06070929726048101
Epoch 9/10, Loss: 0.05364619247829552
Epoch 10/10, Loss: 0.04975969971444057


In [13]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, target in test_loader:
        data = data.view(data.size(0), -1)
        output = model(data)

        # Get the predicted labels
        _, predicted = torch.max(output, 1)

        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = correct / total * 100
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 97.64%
